Projede trafik işaretleri veri kümesi ile CNN modelinin eğitilmesi

Google Drive linki: https://drive.google.com/drive/u/1/folders/1Pe--TRsTpvR1c3mCOXV3SwmBxCT-yyim
---

*İçindekiler:*
* [PyTorch](https://pytorch.org) hakkında
* Trafik işaretlerini tanıması için sinir ağlarının eğitilmesi (MLP + CNN)
* erişilebilir kıyaslama veri kümeleriyle(dataset) çalışma ([Traffic Sign Recognition Benchmark](https://benchmark.ini.rub.de/gtsrb_news.html))

**Porgram çalıştırılırken GPU üzerinden çalıştırınız**

Bu uygulamayı çalıştırmak için bir miktar Python ve numpy bilgisi gerekmektedir. Proram [Google Colab](https://colab.research.google.com/) üzerinde çalıştırılabilir veya tercihe göre Jüpiter not defteri kurup kullanabilirsiniz.

---

## PyTorch Hakkında


### Introduction to PyTorch

["PyTorch Başlangıç" eğitimi](https://pytorch.org/tutorials/beginner/basics/intro.html) bu 9 alt başlık üzerinden ilerler.

*   Temeller
*   Ufak bir başlangıç
*   Tensörler (tensors)
*   Veri Kümeleri (dataset) ve Veri Yükleyici (dataloader)
*   Dönüşümler
*   İnşa Modeli (Build Model)
*   Autograd (PyTorch paketi)
*   Optimizasyon
*   Kaydetme ve Yükleme Modeli

Proje için gerekli kodları Google Colab üzerinde inceleyebilir ve düzenleyebilirsiniz.



## Hazırlıklar ve Eklemeler


Paket Yolu

In [ ]:
# Package Path (this needs to be adapted)
packagePath = "./" # yerel
if 'google.colab' in str(get_ipython()):
  packagePath = "/content/drive/MyDrive/a3-gtsrb/a3-gtsrb/templates"   # Colab

### Gerekli kütüphanelerin eklenmesi (Programın bu kodlarla başlaması gerekmektedir.)

In [ ]:
# os, glob, time, logging
import os, glob, time, logging

# NumPy
import numpy as np

# OpenCV
import cv2

# Matplotlib
import matplotlib.pyplot as plt
# yazıların resimlerin altında görünmesi için
%matplotlib inline

# PyTorch
import torch
import torchvision
import torchvision.transforms as transforms

# Google Colab'a özel paketler
if 'google.colab' in str(get_ipython()):
  # resim görünütleme için
  from google.colab.patches import cv2_imshow


### Gerekli olabilecek yardımcı fonksiyonlar

In [ ]:
def my_imshow(image, windowTitle=None, size=20, depth=3):
  '''
  Displays an image and differentiates between Google Colab and a local Python installation.

  Args:
    image: The image to be displayed

  Returns:
    -
  '''

  if 'google.colab' in str(get_ipython()):
    print(windowTitle)
    cv2_imshow(image)
  else:
    if (size):
      (h, w) = image.shape[:2]
      aspectRatio = float(h)/w
      figsize=(size, size * aspectRatio)
      plt.figure(figsize=figsize)

    if (windowTitle):
      plt.title(windowTitle)

    if (depth == 1):
      plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    elif (depth == 3):
      plt.imshow(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    else:
      plt.imshow(image)
    plt.axis('off')
    plt.show()


### Google Colab kullanımı:
Mount the Google Drive associated with your Google account. You will have to click the authorization link and enter the obtained authorization code here in Colab.

In [ ]:
# Mount Google Drive
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

### PyTorch Eğiticisi ve Deneme Sınıfı

Proje paketinin içinde, `torchHelpers.py` isimli Python kod dosyası ekledik. Bu dosya `Trainer` ve `Tester` isimli iki sınıf bulunduruyor. BUsınıflar sinir ağı eğitim döngülerini ve test kodunu içeriyor.
Bu sınıfların kullanımı şu şekildedir: (Sınıfların kendi dökümantasyonu kullanımları için yeterli bir kaynaktır.):

```
# Train a neural network model
# create a trainer
trainer = Trainer(model, lossFunction, optimizer, device, logLevel=logging.INFO)
# train the model
trainer.train(trainLoader, valLoader, numberOfEpochs)

# Test a neural network model
# create a tester
tester = Tester(model, device, logLevel=logging.INFO)
# test the model
tester.test(testLoader)
```

Hata mesajları ve doğrulamaalar `trainer.metrics` and `tester.metrics` dosyalarını eğittikten sonra çıkacaktır.

In [ ]:
import sys
sys.path.append(packagePath)

from torchHelpers import Trainer, Tester

help(Trainer)

## PyTorch'ta Çok Katmanlı Algılayıcıları Kullanarak Trafik İşareti Sınıflandırması
Bu aşamada çok katmanlı olarak nöral ağı PyTorch ile eğiteceğiz. [Örnek Trafik işareti Veri Kümesi](https://benchmark.ini.rub.de/gtsrb_news.html) kullanılacaktır. Önceden herhangi bir özellik dönüşümü olmayacaktır, girdi ögesi saf piksel verileridir.

### Müsait cihazlardan en uygunu otomatik olarak seçilir (**kod kısım**)

**Colab'da: Çalışma Zamanı(runtime) -> Çalışma zamanı türünü değiştir bölümünde "GPU"yu seçin**

Örnek GPU cihazı:

```
Using device: cuda
Tesla T4
```


İlerleyen aşamada verileri ve modeli bu cihaza kaydedeceğiz. PyTorch aygıt yönetimni otomatik olarak kendisi yapmaktadır (Cuda/ MPS vb için ayar yapmamıza gerek yok.).

In [ ]:
# Check the devices that we have available and prefer CUDA over MPS and CPU
def autoselectDevice(verbose=1):

    # default: CPU
    device = torch.device('cpu')

    if torch.cuda.is_available():
        # CUDA
        device = torch.device('cuda')
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        # MPS (acceleration on Apple silicon M1 / M2 chips)
        device = torch.device('mps')

    if verbose:
        print('Using device:', device)

    # Additional Info when using cuda
    if verbose and device.type == 'cuda':
        print(torch.cuda.get_device_name(0))

    return device

# We transfer our model and data later to this device. If this is a GPU
# PyTorch will take care of everything automatically.
device = autoselectDevice(verbose=1)


### GTSRB Veri Kümesi ile başlangıç (**kod kısım**)

In [ ]:
# GTSRB is available as standard dataset in PyTorch. Nice :)

# Data is loaded and processed in batches of 'batchSize' images
batchSize = 24

# We can add a chain of transforms that is applied to the original data, e.g.
#    resize all images to the same dimensions, e.g. 64x64 pixels
#    convert (batch of) images to a tensor
#    normalize pixel values (to 0-1)

transform = transforms.Compose(
    [transforms.Resize((64, 64)), # resize to 64x64 pixels
     transforms.ToTensor()        # convert to tensor. This will also normalize pixels to 0-1
     ])

# We construct several DataLoaders that take care of loading, storing, caching, pre-fetching the dataset.
# We will have one DataLoader for training, validation and test data.

# Training data
trainSet = torchvision.datasets.GTSRB(root='./data', split='train',
                                      download=True, transform=transform)
trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize,
                                          shuffle=True, pin_memory=True, num_workers=2)
numTrainSamples = len(trainSet)

# Validation and test data
# GTSRB only provides a single test set. To create a validation and test set,
# we split the original GTSRB test set into two parts. The validation set is
# used to tune performance during training. The test set is only used AFTER
# training to evaluation the final performance.

gtsrbTestSet = torchvision.datasets.GTSRB(root='./data', split='test',
                                          download=True, transform=transform)

# Split the original GTSRB test data into 75% used for validation and 25% used for testing
# We do not need to shuffle the data, as we are processing every validation / test image exactly once
length75Percent = int(0.75 * len(gtsrbTestSet))
length25Percent = len(gtsrbTestSet) - length75Percent
lengths = [length75Percent, length25Percent]
valSet, testSet = torch.utils.data.random_split(gtsrbTestSet, lengths)
valLoader = torch.utils.data.DataLoader(valSet, batch_size=batchSize,
                                        shuffle=False, pin_memory=True, num_workers=2)
numValSamples = len(valSet)

testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize,
                                         shuffle=False, pin_memory=True, num_workers=2)
numTestSamples = len(testSet)

# Available traffic sign classes in the dataset
classes = [
    "Speed limit (20km/h)",
    "Speed limit (30km/h)",
    "Speed limit (50km/h)",
    "Speed limit (60km/h)",
    "Speed limit (70km/h)",
    "Speed limit (80km/h)",
    "End of speed limit (80km/h)",
    "Speed limit (100km/h)",
    "Speed limit (120km/h)",
    "No passing",
    "No passing for vehicles over 3.5 metric tons",
    "Right-of-way at the next intersection",
    "Priority road",
    "Yield",
    "Stop",
    "No vehicles",
    "Vehicles over 3.5 metric tons prohibited",
    "No entry",
    "General caution",
    "Dangerous curve to the left",
    "Dangerous curve to the right",
    "Double curve",
    "Bumpy road",
    "Slippery road",
    "Road narrows on the right",
    "Road work",
    "Traffic signals",
    "Pedestrians",
    "Children crossing",
    "Bicycles crossing",
    "Beware of ice/snow",
    "Wild animals crossing",
    "End of all speed and passing limits",
    "Turn right ahead",
    "Turn left ahead",
    "Ahead only",
    "Go straight or right",
    "Go straight or left",
    "Keep right",
    "Keep left",
    "Roundabout mandatory",
    "End of no passing",
    "End of no passing by vehicles over 3.5 metric tons",
]

numClasses = len(classes)

### Veri kümesi sonuçlarını ekrana yazdırır.

In [ ]:
print("Dataset Statistics")
print(f"  # of training samples:   {numTrainSamples}")
print(f"  # of validation samples: {numValSamples}")
print(f"  # of test samples:       {numTestSamples}")
print(f"  # of different classes:  {numClasses}")

### Sonucu görselleştirmek için eklenen kod

In [ ]:
# Visualize a random batch of data from the data set

def imshow(img):
    npimg = img.cpu().numpy() # make sure image is in host memory

    # normalize to 0-1 for visualization
    minPixel = np.min(npimg)
    maxPixel = np.max(npimg)
    npimg = npimg - minPixel
    npimg = npimg / (maxPixel-minPixel)

    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis("off")
    plt.show()


numRows = 8

# get a single random batch of training images
dataIter = iter(trainLoader)
images, labels = next(dataIter)

# print labels
for i in range( batchSize // numRows ):
    print('\n'.join(f'Image {j:2d}: {classes[labels[j]]:5s}' for j in range((i*numRows), (i*numRows)+numRows)))

# show images
imshow(torchvision.utils.make_grid(images, nrow=numRows))

### Neural Network Model Tanımlanması (**kod kısmı**)

[Örnekteki](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html) gibi standart bir "ileri beslemeli (feed-forward)" çok katmanlı algılayıcı tasarlamak istiyoruz. PyTorch terimleriyle, bu alt sınıflarını sınıflandıran ve [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module) `__init__()` ve `forward()` içeren bir Python sınıfıdır. Not olarak [örnek](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)teki `nn.Sequential` kullanmak zorunda değiliz. Katmanları guruplandırmaya yarar ancak gerekli değildir. Her katman ayrı olarak değerlendirilebilir,([örnek](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#define-a-convolutional-neural-network)teki gibi).


proje için aşağıdaki katmanlara ihtiyacımız var: (giriş ve çıkış değerler için)

* Girdi değeri sadece piksel verilerden oluşacaktır, örnek olarak 12288 değeri (64x64x3 olarak tek düzleme işlenmiş görsel)
* [`torch.nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) her katmanda 512, 256, 128, 64 nöron bulunur.
* Çıktı katmanı [`torch.nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) `numClasses` nöronları ile katmanlanır, her sınıf için bir nöron.
* Çıktı katmanındaki nöronlar hariç her nöron [`torch.nn.functional.leaky_relu`](https://pytorch.org/docs/stable/generated/torch.nn.functional.leaky_relu.html#torch.nn.functional.leaky_relu) aktivasyon fonksiyonuna sahiptir.
* **Not olarak: Çıktı katmanında herhangi bir aktivasyon fonksiyonu olmaamlıdır. Hasar hesaplamasında otomatik olarak uygulanacaktır (CrossEntropy kaybı için softmax aktivasyonu).**


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(12288, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 43)

    def forward(self, x):
        x = x.view(-1, 12288)

        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        x = F.leaky_relu(self.fc4(x))
        x = self.fc5(x)
        output = x

        return output


### `torchinfo` paketi kullanılarak oluşturduğumuz ağ modelinin bir özet görüntüsü ekrana yazdırılır. (**kod kısmı**)

Örnek ekran alıntısı:
```
==========================================================================================
Layer (type (var_name))                  Output Shape              Param #
==========================================================================================
Net (Net)                                [1, 43]                   --
├─Linear (fc1)                           [1, 512]                  6,291,968
├─Linear (fc2)                           [1, 256]                  131,328
├─Linear (fc3)                           [1, 128]                  32,896
├─Linear (fc4)                           [1, 64]                   8,256
├─Linear (fc5)                           [1, 43]                   2,795
==========================================================================================
Total params: 6,467,243
Trainable params: 6,467,243
Non-trainable params: 0
Total mult-adds (M): 6.47
==========================================================================================
Input size (MB): 0.05
Forward/backward pass size (MB): 0.01
Params size (MB): 25.87
Estimated Total Size (MB): 25.93
==========================================================================================
```

In [ ]:
# Instatiate our neural network
network = Net()

# Print a summary of the net
%pip install torchinfo

from torchinfo import summary
summary(network, input_size=(1, 3, 64, 64), row_settings=["var_names"])

### Neural Network Eğitimi (kod içeriği)

Modelimizi eğitmek için, bazı tanımlamalar yapmalıyız. Bunlar: loss function, optimizer and hyperparameters. Bunlar modelin eğitim sürecini kontrol eder.

* [`torch.optim.AdamW`](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html?highlight=adamw#torch.optim.AdamW) 'lr=3e-4' olarak ayarlanan öğrenme oranı dışında varsayılan parametrelerle optimize edici olarak kullanılır.
* [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) kayıp fonksiyonu olarak kullanılır. [Dökümantasyonda] (https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) belirtildiği gibi, kayıp hesaplaması sırasında softmax aktivasyonunun uygulanması gerekmektedir.
* Modelin istendiği başarıya ulaşması için yaklaşık 15 deneme gerekmektedir.


`Trainer` sınıfı ile algo-ılayıcı sinir ağını eğitin, sonra  `trainLoader` veri yükleyici olarak ekleyin (veriyi eğitmek için), `valLoader` veriyi doğrulamak için eklenir.

Her deneme için süreç yaklaşık olarak 20 saniyedir(**GPU** üzerinden çalıştığı için kullanılan modele göre değişebilir). Eğitim (doğrulama) verileri 15 eğitimden sonra yaklaşık olarak %95 (%81) olamlıdır .



In [ ]:
##### YOUR CODE GOES HERE ######
import torch.optim as optim
network = Net()

num_epochs = 15
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(network.parameters(), lr=3e-4)

trainer = Trainer(network, criterion, optimizer, device, logLevel=logging.INFO)
trainer.train(trainLoader, valLoader, num_epochs)
################################

### Kayıp ve doğruluk davranışının görselleştirilmesi (**kod kısım**)

`trainer.metrics`'te bulunan verileri kullanarak, aşağıdaki sonuçlara ulaştık:
* Her denemede fonksiyon değeri olarak eğitim kaybı ve doğrulama kaybı.  
* Her denemede fonksiyon değeri olarak eğitim doğruluğu ve doğrulama doğruluğu.  

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Assuming trainer.metrics contains the necessary information

# Plotting training and validation loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(trainer.metrics['epochTrainLoss'], label='Training Loss')
plt.plot(trainer.metrics['epochValLoss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plotting training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(trainer.metrics['epochTrainAccuracy'], label='Training Accuracy')
plt.plot(trainer.metrics['epochValAccuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


### Örnek tahmin değerleri için modelin çeşitli görseller ile çalıştırılması (**kod kısım**)

In [ ]:
# Run some batches of unseen test data through the network and visualize its predictions
numBatches = 4
numRows = 8

 # Iterator through the test DataLoader
dataIter = iter(testLoader)

# for each batch
for batch in range(numBatches):

    # get images and ground truth labels
    images, labels = next(dataIter)

    # push to the device used
    images, labels = images.to(device), labels.to(device)

    # forward pass of the batch of images
    outputs = network(images)

    # find the index of the class with the largest output
    _, predictedLabels         = torch.max(outputs, 1)

    # print labels and outputs
    countCorrect = 0
    for i in range( batchSize // numRows ):
        for j in range((i*numRows), (i*numRows)+numRows):
            print(f'Image {j:2d} - Label: {classes[labels[j]]:5s} | Prediction: {classes[predictedLabels[j]]:5s}')
            if labels[j] == predictedLabels[j]:
                countCorrect = countCorrect + 1

    print(f"\n{(countCorrect / batchSize) * 100.0:.2f}% of test images correctly classified")

    # show images
    imshow(torchvision.utils.make_grid(images))

### Önceden kullanılmamış veri kümelerinde modelin performansının değerlendirilmesi (**kod kısım**)

Eklenen `Tester` sınıfını (aşağıda eklenmiştir) kullanarak modelin önceden kullanılmamış veri kümelerinde  `testLoader` ile test edilmesi. Eğitilmiş model yaklaşık olarak %80 oranında doğruluk payına sahiptir (+- 2% değitim sırasında gerçekleşebilecek rastgele durumlardan dolayı hata payı)

In [ ]:
###### YOUR CODE GOES HERE ######
# Test the network on the final test set
tester = Tester(network, device, logLevel=logging.INFO)
tester.test(testLoader)
#################################

## Convolutional Neural Networks kullanılarak Trafik İşareti Sınıflandırmanın  PyTorch ile gerçekleştirilmesi

Bu kısımda CNN modelini pyTorch kullanarak [German Traffic Sign Recognition Benchmark](https://benchmark.ini.rub.de/gtsrb_news.html) veri kümesi ile eğitilecektir. Önceden herhangi bir özellik dönüşümü olmayacaktır, yani saf piksel değerleri girdi olacaktır.

**Bunun için, aynı veri kümeleri, loss fonksiyonu, optimizer, hiperparametreler, 'Trainer' ve 'Test' sınıfı ile yukarıdaki işlemler tekrarlanabilir. Sadece farklı bir ağ modeli tanımlanmalı ve ağ parametrelerinin giriş olarak optimize edilmesi gerektirdiğinden optimize edicinin yeniden oluşturulması gerekir.**

### Convolutional Neural Network tanımlanması (**kod ksıım**)

[Örnekteki](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) gibi bir CNN modeli tanımlamak için ağ sınıfımızın genel yapısı, çok katmanlı algılayanlar için öncekiyle aynıdır. Bununla birlikte, ek katmanlar (evrişim(convolutional) ve havuzlama(pooling) katmanları) kullanacağız.

Girdi ve çıktı değerleri için aşağıdaki katmanları kullanmamız gerekiyor:

* Giriş ham piksel değerleri, yani 64x64x3 (**çok katmanlı algılama için düzleştirilmez**) olacaktır.
* 1 [`torch.nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d) 16 çıkış kanalı ile evrişimsel katman (farklı özellikler, 3x3 çekirdek (kernel) ve `padding='same'`
* 1 [`torch.nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d) 16 çıkış kanalı ile evrişimsel katman (farklı özellikler, 3x3 çekirdek (kernel) ve `padding='same'`
* 1 [`torch.nn.MaxPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d) 2x2 çekirdek ile havuzlama(pooling) katmanı
* 1 [`torch.nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d) 32 çıkış kanalı ile evrişimsel katman (farklı özellikler, 3x3 çekirdek (kernel) ve `padding='same'`
* 1 [`torch.nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d) 32 çıkış kanalı ile evrişimsel katman (farklı özellikler, 3x3 çekirdek (kernel) ve `padding='same'`
* 1 [`torch.nn.MaxPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d) 2x2 çekirdek ile havuzlama(pooling) katmanı
* 1 [`torch.nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d) 64 çıkış kanalı ile evrişimsel katman (farklı özellikler), 3x3 çekirdek (kernel) ve `padding='same'`
* 1 [`torch.nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d) 64 çıkış kanalı ile evrişimsel katman (farklı özellikler), 3x3 çekirdek (kernel) ve `padding='same'`
* 1 [`torch.nn.MaxPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d) 2x2 çekirdek ile havuzlama(pooling) katmanı
* 1 [`torch.nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) 64 nöron ile tam erişimli katman
* Çıktı katmanı [`torch.nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) `numClasses` nöronları ile tamamiyle bağlıdır, her sınıf için bir nöron.
* Çıkış katmanındaki nöronlar ve havuz katmanları hariç tüm nöronlar [`torch.nn.functional.leaky_relu`](https://pytorch.org/docs/stable/generated/torch.nn.functional.leaky_relu.html#torch.nn.functional.leaky_relu) aktivasyon fonksiyonuna sahiptir.
* **Önemli: Çıkış katmanının herhangi bir aktivasyon işlevi olmamalıdır. Kayıp hesaplaması sırasında otomatik olarak uygulanacaktır(CrossEntropy kaybı için softman aktivasyonu).**

Model özeti için örnek ekran çıktısı:
```
==========================================================================================
Layer (type (var_name))                  Output Shape              Param #
==========================================================================================
ConvNet (ConvNet)                        [1, 43]                   --
├─Conv2d (conv1)                         [1, 16, 64, 64]           448
├─Conv2d (conv2)                         [1, 16, 64, 64]           2,320
├─MaxPool2d (pool)                       [1, 16, 32, 32]           --
├─Conv2d (conv3)                         [1, 32, 32, 32]           4,640
├─Conv2d (conv4)                         [1, 32, 32, 32]           9,248
├─MaxPool2d (pool)                       [1, 32, 16, 16]           --
├─Conv2d (conv5)                         [1, 64, 16, 16]           18,496
├─Conv2d (conv6)                         [1, 64, 16, 16]           36,928
├─MaxPool2d (pool)                       [1, 64, 8, 8]             --
├─Linear (fc1)                           [1, 64]                   262,208
├─Linear (fc2)                           [1, 43]                   2,795
==========================================================================================
Total params: 337,083
Trainable params: 337,083
Non-trainable params: 0
Total mult-adds (M): 40.01
==========================================================================================
Input size (MB): 0.05
Forward/backward pass size (MB): 1.84
Params size (MB): 1.35
Estimated Total Size (MB): 3.23
==========================================================================================
```


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding='same')
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, padding='same')
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding='same')
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding='same')
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv5 = nn.Conv2d(32, 64, kernel_size=3, padding='same')
        self.conv6 = nn.Conv2d(64, 64, kernel_size=3, padding='same')
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.fc1 = nn.Linear(64 * 8 * 8, 64)
        self.fc2 = nn.Linear(64, 43)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        x = self.pool1(x)

        x = F.leaky_relu(self.conv3(x))
        x = F.leaky_relu(self.conv4(x))
        x = self.pool2(x)

        x = F.leaky_relu(self.conv5(x))
        x = F.leaky_relu(self.conv6(x))
        x = self.pool3(x)

        x = x.view(-1, 64 * 8 * 8)
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [ ]:
# Instantiate the CNN
cnn = ConvNet()
from torchinfo import summary
summary(cnn, input_size=(1, 3, 64, 64), row_settings=["var_names"])

CNN modelini veri kümesinde eğittikten sonra önceki eğitimlerde kullanılmamış bir veri kümesi ile doğruluğunu test edin

Modelin eğitimi her deneme için ortalama 20 saniye sürmelidir (**kullanılan GPU'ya göre süre değişebilir**). Yaklaşık olarak 15 deneme sonucunda modelin doğruluğu hatasız olmaya yaklaşmaktadır. Yeni ve daha önce eğitimde kullanılmamş (modelin eğitilmediği) bir veri kümesi için bu oran %90 civarındadır (15 eğitim sonrası).

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Instantiate the CNN
cnn = ConvNet()
num_epochs = 15
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(cnn.parameters(), lr=3e-4)
trainer = Trainer(cnn, criterion, optimizer, device, logLevel=logging.INFO)
trainer.train(trainLoader, valLoader, num_epochs)

# Test the network on the final test set



# ***Kaynakça***

[1] https://kaanugurluoglu123.medium.com/nesne-tan%C4%B1ma-algoritmas%C4%B1-faster-r-cnn-nedir-1738f0cca8b7

[2] https://ieeexplore.ieee.org/abstract/document/8756165

[3] https://www.academia.edu/87657329/CNN_MODEL_FOR_TRAFFIC_SIGN_RECOGNITION?source=swp_share

[4] https://www.youtube.com/watch?v=SWaYRyi0TTs&pp=ygUXY25uIG1vZGVsIGZvciB0cmFmZmljIHM%3D

[5] https://www.youtube.com/watch?v=eS8sE1M9dks&pp=ygUXY25uIG1vZGVsIGZvciB0cmFmZmljIHM%3D

[6] https://www.youtube.com/watch?v=4KEojbSJy4Y

[7] https://www.youtube.com/watch?v=QzY57FaENXg&pp=ygUXY25uIG1vZGVsIGZvciB0cmFmZmljIHM%3D

[8] https://www.diva-portal.org/smash/get/diva2:1575390/FULLTEXT02.pdf

[9] https://pyimagesearch.com/2019/11/04/traffic-sign-classification-with-keras-and-deep-learning/

[10] https://github.com/ItsCosmas/Traffic-Sign-Classification
